In [1]:
#tensorいじりのテストを行う
import torch
import torch.nn
x = torch.randn(10,10,10)
print(torch.cuda.is_available())
from audio_to_multiple_pose_gan.dataset import generate_batch, get_processor
from audio_to_multiple_pose_gan.config import get_config

True


In [3]:
import pandas as pd
from common.consts import AUDIO_SHAPE
batch_size = 32
train_csv: str = "~/Desktop/AI-Experiments/speechandtext2gesture/Gestures/train.csv"
#df = pd.read_csv(self.args.train_csv)
#cfg = get_config(self.args.config)
df = pd.read_csv(train_csv)
#cfg = get_config(config)
cfg: dict = {"processor": "audio_to_pose", "input_shape": [None, AUDIO_SHAPE]}
process_row, decode_pose = get_processor(cfg)

In [4]:
from common.consts import AUDIO_SHAPE
configs = {
    "audio_to_pose": {"num_keypoints": 98, "processor": "audio_to_pose", "flatten": False, "input_shape": [None, AUDIO_SHAPE]},
    "audio_to_pose_inference": {"num_keypoints": 98, "processor": "audio_to_pose_inference", "flatten": False, "input_shape": [None, AUDIO_SHAPE]}
}
cfg: dict = {"processor": "audio_to_pose", "input_shape": [None, AUDIO_SHAPE]}
process_row, decode_pose = get_processor(cfg)

In [5]:
row = df.sample(n=1).iloc[0]
X, Y = generate_batch(df, process_row, batch_size)

In [6]:
X

array([[-0.06668285, -0.06729475, -0.05482565, ..., -0.13014463,
        -0.1219707 , -0.11283086],
       [-0.0094066 , -0.00675579, -0.00532913, ...,  0.01945134,
         0.01724727,  0.01056778],
       [-0.07300902, -0.10317692, -0.07244992, ...,  0.03822149,
         0.04304965,  0.03956992],
       ...,
       [ 0.28201774,  0.24430291,  0.19337395, ...,  0.00428943,
         0.00292572, -0.00142749],
       [-0.03214574, -0.04217231, -0.07534076, ..., -0.12751943,
        -0.10872138, -0.08829635],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)